In [1]:
import pymysql
import requests
import pandas as pd
from bs4 import BeautifulSoup
from datetime import datetime
import time
import pickle
import numpy as np
import re
import ast

In [2]:
import os
import toml

if os.name == "posix":  # Linux, macOS
    import mysql.connector  # Usar MySQL en Linux, macOS
elif os.name == "nt":  # Windows
    import pymysql  # Usar PyMySQL en Windows
else:
    raise Exception("Sistema operativo no soportado")

config = toml.load("../.streamlit/secrets.toml")
db_config = config["database"]
database = db_config["database"]

# Conectar a MySQL
if os.name == "posix":  # Linux/macOS
    conn = mysql.connector.connect(
        host=db_config["host"],
        user=db_config["user"],
        password=db_config["password"],
        database=database
    )
elif os.name == "nt":  # Windows
    conn = pymysql.connect(
        host=db_config["host"],
        user=db_config["user"],
        password=db_config["password"],
        database=database,
        cursorclass=pymysql.cursors.DictCursor
    )

cursor = conn.cursor()

# Consulta obtener el último id_tec
query_id = """
SELECT id_oferta 
FROM ofertas 
WHERE id_oferta LIKE "id_tec_%" 
ORDER BY CAST(SUBSTRING_INDEX(id_oferta, "_", -1) AS UNSIGNED) DESC 
LIMIT 1;
"""
cursor.execute(query_id)

if os.name == "posix":
    ultimo_id_man = cursor.fetchone()[0]
else:
    ultimo_id_man = cursor.fetchone()["id_oferta"]

# Consulta obtener las URLs
query_urls = """
SELECT url 
FROM ofertas 
WHERE url LIKE "https://www.tecnoempleo.com%";
"""
cursor.execute(query_urls)

urls_existent = [url[0] for url in cursor.fetchall()]

cursor.close()
conn.close()

Último ID Tecnoempleo: id_tec_3604
DataFrame de Idiomas:
  id_oferta  idioma  nivel
0  id_tec_1  Inglés  Medio
1  id_tec_2  Inglés   Alto
2  id_tec_3  Alemán   Alto
3  id_tec_3  Inglés   Alto
4  id_tec_4    None   None
DataFrame de Habilidades:
                       habilidad  hab_id
0           Pensamiento creativo       1
1  Tolerancia a la incertidumbre       2
2   Resistencia a la frustración       3
3            Comunicación verbal       4
4           Capacidad de escucha       5
URLs Existentes:
['https://www.tecnoempleo.com/arquitecto-soluciones-observabilidad-npr-spain/elk-grafana-python-linux-aw/rf-f00d1747f20d039c014e', 'https://www.tecnoempleo.com/ios-native-developers-page-personnel/ios-swift-java/rf-9257150d6295d33e384c', 'https://www.tecnoempleo.com/dba-oracle-oracle-exadata-seltime/oracle-pl-sql-etl/rf-50cc1782c2a8737a6145', 'https://www.tecnoempleo.com/tecnico-microinformatico-con-fp-grupo-sermicro/windows-office-ticketing-so/rf-8f7f1e1db285f30be94a', 'https://www.tecn

/tmp/ipykernel_6404/2763306181.py:48: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_idiomas_original = pd.read_sql(query_idiomas, conn)
/tmp/ipykernel_6404/2763306181.py:52: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_habilidades_original = pd.read_sql(query_habilidades, conn)


In [6]:
def extraer_urls(urls_existent, num_paginas=None):

    base_url = "https://www.tecnoempleo.com/ofertas-trabajo/?pagina={}"
    nuevas_urls = [] 
    nombres_empresas = [] 
    ciudades = []
    pagina = 1

    while True:
        url = base_url.format(pagina)
        response = requests.get(url)

        if response.status_code != 200:
            print(f"Error al acceder a la página {pagina}")
            break

        soup = BeautifulSoup(response.text, "html.parser")
        enlaces = soup.find_all("a", class_="font-weight-bold text-cyan-700")
        urls = [enlace.get("href") for enlace in enlaces if enlace.get("href")]
        if not urls:
            print(f"No se encontraron más ofertas en la página {pagina}. Finalizando.")
            break
        for url in urls:
            if url not in urls_existent:
                nuevas_urls.append(url)

            # Extraemos los nombres de las empresas
            nombre_empresas = soup.find_all("a", class_="text-primary link-muted")
            for empresa in nombre_empresas:
                nombres_empresas.append(empresa.text.strip())
            for div in soup.find_all("div", class_="col-12 col-lg-3 text-gray-700 pt-2 text-right hidden-md-down"):
                if div.b:
                    ciudades.append(div.b.text.strip())
                else:
                    ciudades.append("No especificado")
        print(f"Página {pagina}: {len(nuevas_urls)} URLs añadidos.") # Hacemos print para asegurarnos de que está funcionando correctamente.
        
        # Calculamos el número total de páginas para asegurarnos de que las recorra todas:
        if num_paginas is None:
            total_ofertas = int(soup.find("h1").text.split()[0].replace(".", ""))
            total_paginas = (total_ofertas // 30) + (1 if total_ofertas % 30 != 0 else 0)
            if pagina >= total_paginas:
                break
        elif pagina >= num_paginas:
            break

        pagina +=1        

        if not urls:
            print(f"No se encontraron más ofertas en la página {pagina}. Finalizando.")
            break


    return nuevas_urls, nombres_empresas, ciudades

# Hacemos una función que recorre los urls para extraer los datos:
def extraer_datos_de_urls(nuevas_urls, nombres_empresas, ciudades):
    todas_las_ofertas = []
    todas_las_tecnologias = set()

    # Creamos un bucle que recorra las urls:
    for idx, link in enumerate(nuevas_urls):
        response_link = requests.get(link)
        if response_link.status_code != 200:
            print(f"Error al acceder a {link}")
            continue

        soup_link = BeautifulSoup(response_link.text, "html.parser")

        # Sacamos título, fecha, habilidades y añadimos el timestamp para que se indique la fecha en la que se han solicitado los datos. 
        titulo = soup_link.find("h1").get_text(strip=True) if soup_link.find("h1") else "Sin título"
        fecha = soup_link.find("span", class_="ml-4").get_text(strip=True) if soup_link.find("span", class_="ml-4") else "Sin fecha"
        habilidades = [x.text.strip() for x in soup_link.find_all("div", class_="d-flex py-2")[1].find_all("a")]
        timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")

        # Tecnologías. En el caso de tecnologías, limpiamos ya la información que sacamos para que nos muestre solola tecnología 
        # sin el texto que no es necesario: "Ofertas de Empleo de":
        div = soup_link.find("div", class_="pl--12 pr--12")
        tecnologias = [a["title"].replace("Ofertas de Empleo de ", "").lower() for a in div.find_all("a") if div and a.has_attr("title")]
        todas_las_tecnologias.update(tecnologias)

        # Extraemos todos los datos principales
        todos_textos_enunciados = [span.get_text(strip=True) for span in soup_link.find_all("span", class_="d-inline-block px-2")]
        todos_textos = [span.get_text(strip=True) for span in soup_link.find_all("span", class_="float-end")]
        diccionario_principales = dict(zip(todos_textos_enunciados, todos_textos))

        # Extraemos otros detalles de la oferta
        todos_detalles = soup_link.find_all("p", class_="m-0")
        todos_textos_detalles = [x.text.strip() for x in todos_detalles if len(x["class"]) == 1][1:]
        diccionario_detalles = {}
        for item in todos_textos_detalles:
            if ":" in item:
                clave, valor = item.split(":", 1)
                diccionario_detalles[clave.strip()] = valor.strip()
            else:
                diccionario_detalles[item.strip()] = None

        # Creamos el diccionario para añadir toda la información
        oferta = {
            "Título": titulo,
            "URL": link,
            "Tecnologías": tecnologias,
            "Time Stamp": timestamp,
            "Fecha publicacion": fecha,
            "Habilidades": habilidades,
            "Nombre de empresa": nombres_empresas[idx] if idx < len(nombres_empresas) else "No encontrado",
            "Ciudad": ciudades[idx] if idx < len(ciudades) else "No especificado",  # # Nos aseguramos de que los nombres de empresas y 
            #las ciudades se corresponden con los urls sacados
        }

        # Creamos un bucle para asegurarnos de que añadimos todos los datos y, en el caso de que no haya datos, que se añadan también. 
        for clave, valor in diccionario_principales.items():
            oferta[clave] = valor if valor else "No especificado"

        for clave, valor in diccionario_detalles.items():
            oferta[clave] = valor if valor else "No especificado"

        todas_las_ofertas.append(oferta)
        print(f"Procesada oferta: {titulo}")
        print(len(todas_las_ofertas))

    # Creamos el DF
    df_ofertas = pd.DataFrame(todas_las_ofertas)

    # Creamos columnas binarias para cada tecnología. Esto nos facilita hacer un DF para más adelante relacionarlo con el DF principal. 
    for tecnologia in todas_las_tecnologias:
        df_ofertas[tecnologia] = df_ofertas["Tecnologías"].apply(lambda x: 1 if tecnologia in x else 0)
    df_ofertas.drop(columns=["Tecnologías"], inplace=True)

    return df_ofertas

# Llamamos a las funciones
nuevas_urls, nombres_empresas, ciudades = extraer_urls(urls_existent)
df_ofertas = extraer_datos_de_urls(nuevas_urls, nombres_empresas, ciudades)
df_ofertas.to_pickle("/Pickles/ofertas_nuevas_tecnoempleo.pkl") 
print(df_ofertas)

Página 1: 22 URLs añadidos.
Página 2: 50 URLs añadidos.
Página 3: 63 URLs añadidos.
Página 4: 87 URLs añadidos.
Página 5: 110 URLs añadidos.
Página 6: 136 URLs añadidos.
Página 7: 166 URLs añadidos.
Página 8: 196 URLs añadidos.
Página 9: 211 URLs añadidos.
Página 10: 215 URLs añadidos.
Página 11: 223 URLs añadidos.
Página 12: 233 URLs añadidos.
Página 13: 245 URLs añadidos.
Página 14: 257 URLs añadidos.
Página 15: 274 URLs añadidos.
Página 16: 287 URLs añadidos.
Página 17: 303 URLs añadidos.
Página 18: 323 URLs añadidos.
Página 19: 350 URLs añadidos.
Página 20: 372 URLs añadidos.
Página 21: 388 URLs añadidos.
Página 22: 400 URLs añadidos.
Página 23: 407 URLs añadidos.
Página 24: 422 URLs añadidos.
Página 25: 431 URLs añadidos.
Página 26: 440 URLs añadidos.
Página 27: 452 URLs añadidos.
Página 28: 461 URLs añadidos.
Página 29: 463 URLs añadidos.
Página 30: 468 URLs añadidos.
Página 31: 475 URLs añadidos.
Página 32: 488 URLs añadidos.
Página 33: 505 URLs añadidos.
Página 34: 533 URLs aña

/tmp/ipykernel_6404/2946741327.py:127: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_ofertas[tecnologia] = df_ofertas["Tecnologías"].apply(lambda x: 1 if tecnologia in x else 0)
/tmp/ipykernel_6404/2946741327.py:127: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_ofertas[tecnologia] = df_ofertas["Tecnologías"].apply(lambda x: 1 if tecnologia in x else 0)
/tmp/ipykernel_6404/2946741327.py:127: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor 

                                            Título  \
0      UrgenteTécnico Senior Aplicaciones BBDD SQL   
1            UrgenteSenior Python Backend Engineer   
2    UrgenteTécnico Sistemas de Telecomunicaciones   
3                 Analista Pro. COBOL CICS DB2 JCL   
4                           Programadores React.JS   
..                                             ...   
659                   Consultor/ Programador Meta4   
660            Gestor/a Mantenimiento - Facilities   
661            Ingeniero/a de software .Net Senior   
662                     Tecnico Sistemas Senior N2   
663                           Senior GNSS Engineer   

                                                   URL           Time Stamp  \
0    https://www.tecnoempleo.com/tecnico-senior-apl...  2025-03-06 02:41:20   
1    https://www.tecnoempleo.com/senior-python-back...  2025-03-06 02:41:21   
2    https://www.tecnoempleo.com/tecnico-sistemas-t...  2025-03-06 02:41:21   
3    https://www.tecnoempleo.com/an

/tmp/ipykernel_6404/2946741327.py:127: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_ofertas[tecnologia] = df_ofertas["Tecnologías"].apply(lambda x: 1 if tecnologia in x else 0)
/tmp/ipykernel_6404/2946741327.py:127: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_ofertas[tecnologia] = df_ofertas["Tecnologías"].apply(lambda x: 1 if tecnologia in x else 0)
/tmp/ipykernel_6404/2946741327.py:127: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor 

In [7]:
extraccion = "/Pickles/ofertas_nuevas_tecnoempleo.pkl"

In [8]:
def limpieza_tecnoempleo_actualizacion(extraccion, ultimo_id_tec):

    ultimo_numero = int(ultimo_id_tec.split("_")[-1])  # Obtiene el número final del ID
    
    df_ofertas = pd.read_pickle(extraccion)
    
    # Generar nuevos IDs a partir del siguiente número
    df_ofertas["id"] = ["id_tec_" + str(i) for i in range(ultimo_numero + 1, ultimo_numero + 1 + len(df_ofertas))]
    
    # Reordenar la columna "id" al inicio
    columna_extraida_ = df_ofertas.pop("id")
    df_ofertas.insert(0, "id", columna_extraida_)
    
    
    #Generamos df matricial de tecnologías, lo pasamos a pickle y eliminamos las columnas
    df_tecnologias_matricial = df_ofertas[[df_ofertas.columns[0]] + list(df_ofertas.columns[25:])]
    lista_tecnologias = list(df_ofertas.columns[25:])
    df_ofertas.drop(columns=lista_tecnologias, inplace=True) 
    
    
    # Renombramos la columna con título vacío (columna con información de trabajo remoto o no):
    df_ofertas.rename(columns={"": "Teletrabajo"}, inplace=True)
    
    # De la información que nos quedamos en la columna de Teletrabajo, unificamos los valores por: 100% en remoto, híbrido y presencial.
    df_ofertas["Teletrabajo"] = df_ofertas["Teletrabajo"].apply(lambda x: "Remoto" if "Remoto" in x or "Teletrabajo" in x else "Híbrido" if "Híbrido" in x else "Presencial")
    
    # Verificamos los resultados
    df_ofertas["Teletrabajo"].unique()
    
    # Limpiamos la columna de Fecha de publicación, eliminando el texto que nos sobra y convirtiendo a datetime:
    df_ofertas["Fecha publicacion"] = df_ofertas["Fecha publicacion"].str.replace("Actualizada", "").str.replace("Nueva", "")
    df_ofertas["Fecha publicacion"] = pd.to_datetime(df_ofertas["Fecha publicacion"], format="%d/%m/%Y")
    
    # Quitamos los caracteres no deseados de la columna Idiomas y dividimos la columna en dos: una para el idioma y otra para el nivel.
    df_ofertas["Idiomas"] = df_ofertas["Idiomas"].fillna("")
    df_ofertas["Idiomas"] = df_ofertas["Idiomas"].str.replace(r"[\t\r\n|]", "", regex=True)
    
    # Extraemos todos los Idiomas y niveles
    def extraer_Idiomas_niveles(texto):
        if not texto.strip():  
            return np.nan, np.nan
        matches = re.findall(r"(\w+)\s*\((\w+)\)", texto)
        if not matches:
            return np.nan, np.nan 
        Idiomas = [match[0] for match in matches]
        niveles = [match[1] for match in matches]
    
        # Unimos los Idiomas y niveles en cadenas separadas por comas
        return ", ".join(Idiomas), ", ".join(niveles)
    
    
    df_ofertas[["Idiomas_limpios", "niveles_limpios"]] = df_ofertas["Idiomas"].apply(extraer_Idiomas_niveles).apply(pd.Series)
    
    
    df_ofertas["Idiomas_limpios"] = df_ofertas["Idiomas_limpios"].replace("", np.nan)
    df_ofertas["niveles_limpios"] = df_ofertas["niveles_limpios"].replace("", np.nan)
    
    
    df_ofertas.drop(columns=["Idiomas"], inplace=True)
    
    # Dataframe solo con id, idiomas y niveles limpios
    df_idiomas1 = df_ofertas[["id","Idiomas_limpios","niveles_limpios"]]
    
    # Por cada idioma y nivel, creamos un diccionario con el id, idioma y nivel y los añade a una lista
    dict_list = []
    for i, row in df_idiomas1.iterrows():
        if pd.notna(row["Idiomas_limpios"]) and pd.notna(row["niveles_limpios"]):
            idiomas = row["Idiomas_limpios"].split(", ")
            niveles = row["niveles_limpios"].split(", ")
            for idioma, nivel in zip(idiomas, niveles):
                dict_list.append({"id": row["id"], "idioma": idioma, "nivel": nivel})
        else:
            dict_list.append({"id": row["id"], "idioma": None, "nivel": None})
    
    df_ofertas.drop(columns = ["Idiomas_limpios", "niveles_limpios"], inplace = True)
    # Convertimos la lista de diccionarios a un dataframe
    df_idiomas = pd.DataFrame(dict_list)
    
    df_idiomas_original.rename(columns={"id_oferta": "id"}, inplace=True)
    
    df_idiomas_actualizado = pd.concat([df_idiomas, df_idiomas_original], ignore_index=True)
    
    
    # Limpiamos la columna de Salario y la unificamos a bruto por año.
    
    # Separamos las columnas para crear una con la unidad reflejada (mes, año, hora):
    split_data = df_ofertas["Salario"].str.split(" Bruto/", expand=True)
    df_ofertas["Rango"] = split_data[0]  
    df_ofertas["Unidad"] = split_data[1] 
    
    # Dividimos los valores para hacer dos columnas de mínimo y máximo:
    rango_split = df_ofertas["Rango"].str.split("-\xa0", expand=True)
    df_ofertas["Min"] = rango_split[0].str.replace("€", "").str.replace(".", "").str.strip().astype(float)
    df_ofertas["Max"] = rango_split[1].str.replace("€", "").str.replace(".", "").str.strip().astype(float)
    
    # Usamos where para cambiar la palabra mes por 12 y hora por 2080 (40h a la semana por 52 semanas):
    df_ofertas["Factor"] = np.where(
        df_ofertas["Unidad"] == "mes", 12,
        np.where(df_ofertas["Unidad"] == "hora", 2080, 1)
    )
    # Multiplicamos las columnas y dividimos por mil para una mejor visualización:
    df_ofertas["salario_desde"] = df_ofertas["Min"] * df_ofertas["Factor"] / 1000
    df_ofertas["salario_hasta"] = df_ofertas["Max"] * df_ofertas["Factor"] / 1000
    
    df_ofertas.drop(columns=["Rango", "Unidad", "Min", "Max", "Factor"], inplace=True)
    
    # Para la información de la columna de habilidades, creamos un nuevo DF:
    df_habilidades_matricial_tecnoempleo_actualizacion = df_ofertas[["id", "Habilidades"]].copy()
    
    # Transformamos de string a lista: 
    #df_habilidades_matricial_tecnoempleo_actualizacion["Habilidades"] = df_habilidades_matricial_tecnoempleo_actualizacion["Habilidades"].apply(ast.literal_eval)
    
    # Creamos una lista con las habilidades sin duplicar:
    todas_habilidades = sum(df_habilidades_matricial_tecnoempleo_actualizacion["Habilidades"], [])
    habilidades_unicas = list(set(todas_habilidades))
    
    # Creamos columnas binarias para cada habilidad:
    for habilidad in habilidades_unicas:
        df_habilidades_matricial_tecnoempleo_actualizacion[habilidad] = df_habilidades_matricial_tecnoempleo_actualizacion["Habilidades"].apply(lambda x: 1 if habilidad in x else 0)
    
    df_habilidades_matricial_tecnoempleo_actualizacion.drop(columns=["Habilidades"], inplace=True)
    
    
    
    
    #Imprescindible residir, cambiamos los valores para unicficarlos:
    df_ofertas["Imprescindible Residir"] = df_ofertas["Imprescindible Residir"].replace(
        {"España": "País Puesto", "Spain": "País Puesto", "Country": "País Puesto", 
         "Not Required": "No requerido"})
    df_ofertas["Imprescindible Residir"]
    
    # Vamos a duplicar las filas por ID, para asociar todas las provincias relacionadas con ese ID y hacemos un nuevo DF:
    df_provincias = df_ofertas[["id", "Otras Provincias"]]
    
    df_provincias["Otras Provincias"] = df_provincias["Otras Provincias"].apply(lambda x: str(x).split(", ") if pd.notna(x) else [None])
    
    df_provincias = df_provincias.explode("Otras Provincias").reset_index(drop=True)
    
    
    df_ofertas.drop(columns=["Otras Provincias"], inplace=True)
    
    
    df_ofertas.to_pickle("/Pickles/general_tecnoempleo_limpio.pkl")
    df_habilidades_matricial_tecnoempleo_actualizacion.to_pickle("/Pickles/habilidades_matricial_tecnoempleo_actualizacion.pkl")
    df_idiomas_actualizado.to_pickle("/Pickles/idiomas_actualizacion.pkl")
    df_tecnologias_matricial.to_pickle("/Pickles/tecnologias_matricial_tecnoempleo_actualizacion.pkl")
    df_provincias.to_pickle("/Pickles/provincias.pkl")

    return df_ofertas, df_provincias

In [9]:
limpieza_tecnoempleo_actualizacion(extraccion, ultimo_id_tec)

/tmp/ipykernel_6404/2879448358.py:8: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_ofertas["id"] = ["id_tec_" + str(i) for i in range(ultimo_numero + 1, ultimo_numero + 1 + len(df_ofertas))]
/tmp/ipykernel_6404/2879448358.py:12: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_ofertas.insert(0, "id", columna_extraida_)
/tmp/ipykernel_6404/2879448358.py:135: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the

(              id                                         Título  \
 0    id_tec_3605    UrgenteTécnico Senior Aplicaciones BBDD SQL   
 1    id_tec_3606          UrgenteSenior Python Backend Engineer   
 2    id_tec_3607  UrgenteTécnico Sistemas de Telecomunicaciones   
 3    id_tec_3608               Analista Pro. COBOL CICS DB2 JCL   
 4    id_tec_3609                         Programadores React.JS   
 ..           ...                                            ...   
 659  id_tec_4264                   Consultor/ Programador Meta4   
 660  id_tec_4265            Gestor/a Mantenimiento - Facilities   
 661  id_tec_4266            Ingeniero/a de software .Net Senior   
 662  id_tec_4267                     Tecnico Sistemas Senior N2   
 663  id_tec_4268                           Senior GNSS Engineer   
 
                                                    URL           Time Stamp  \
 0    https://www.tecnoempleo.com/tecnico-senior-apl...  2025-03-06 02:41:20   
 1    https://www.tecn